In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [30]:
df =pd.read_csv('C:\\Users\Ente_LJS\\Anaconda_src\\Semi_Project_1pick\\train.csv')

In [31]:
# df_h = df.loc[df["parentesco1"]==1]
df_h = df_h.fillna(0)
df_y =pd.DataFrame(df_h["Target"])
df_y.head()

,Target
0,4
1,4
2,4
5,4
8,4


In [32]:
#불필요한 피쳐들 제거해 x1값 만들기 
# age, SQBescolari, SQBage, SQBhogar_total, SQBedjefe, SQBhogar_nin, SQBovercrowding,
# SQBdependency, SQBmeaned, agesq

#결과값 y에 해당하는 값 삭제 
df_x1 = df_h.drop(["Target"],1)
#세대주성별 교육년수 (논의필요)
df_x1 = df_x1.drop(["edjefa", "edjefe"],1) 
#중복정보 제거
df_x1 = df_x1.drop(["dependency","female","area2","hacdor","hacapo","bedrooms","r4h3","r4m3"],1) 
#수학적으로 의미가 없는 값 제거
df_x1 = df_x1.drop(["Id","SQBescolari", "SQBage", "SQBhogar_total", "SQBedjefe", "SQBhogar_nin", "SQBovercrowding","idhogar"],1) 
#세대주와의 관계열 제거
df_x1 = df_x1.drop(["parentesco1","parentesco2","parentesco3","parentesco4","parentesco5","parentesco6","parentesco7","parentesco8",
                   "parentesco9","parentesco10","parentesco11","parentesco12"],1)
#집세 임시제거!!!!!!!!!!!!!!!!!!!!!!!!!!
df_x1 = df_x1.drop(["v2a1"],1)
df_x1.head()

,rooms,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4m1,r4m2,r4t1,...,lugar2,lugar3,lugar4,lugar5,lugar6,area1,age,SQBdependency,SQBmeaned,agesq
0,3,1,1,0,0.0,0,1,0,0,0,...,0,0,0,0,0,1,43,0.0,100.0,1849
1,4,1,1,1,1.0,0,1,0,0,0,...,0,0,0,0,0,1,67,64.0,144.0,4489
2,8,1,1,0,0.0,0,0,0,1,0,...,0,0,0,0,0,1,92,64.0,121.0,8464
5,5,1,1,1,1.0,0,2,1,1,1,...,0,0,0,0,0,1,38,1.0,121.0,1444
8,2,1,1,0,0.0,0,1,2,1,2,...,0,0,0,0,0,1,30,1.0,100.0,900


In [33]:
#새로운열 추가 
# df_x1['rent_to_rooms'] = df_x1['v2a1']/df_x1['rooms']
# df_x1['r4t3_to_rooms'] = df_x1['r4t3']/df_x1['rooms']
# df_x1['rent_to_r4t3'] = df_x1['v2a1']/df_x1['r4t3']
# df_x1['v2a1_to_r4t3'] = df_x1['v2a1']/(df_x1['r4t3'] - df_x1['r4t1'])
df_x1['lent'] = df_x1['tamviv']-df_x1['tamhog']

#열 생성 이후 불필요한 열 제거 
df_x1 = df_x1.drop(["r4t1","r4t2","r4t3","tamhog","tamviv"],1) 
df_x1.head()

,rooms,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4m1,r4m2,escolari,...,lugar3,lugar4,lugar5,lugar6,area1,age,SQBdependency,SQBmeaned,agesq,lent
0,3,1,1,0,0.0,0,1,0,0,10,...,0,0,0,0,1,43,0.0,100.0,1849,0
1,4,1,1,1,1.0,0,1,0,0,12,...,0,0,0,0,1,67,64.0,144.0,4489,0
2,8,1,1,0,0.0,0,0,0,1,11,...,0,0,0,0,1,92,64.0,121.0,8464,0
5,5,1,1,1,1.0,0,2,1,1,11,...,0,0,0,0,1,38,1.0,121.0,1444,0
8,2,1,1,0,0.0,0,1,2,1,9,...,0,0,0,0,1,30,1.0,100.0,900,0


In [34]:
df_y["Target"]=df_y["Target"]-1

In [8]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [9]:
def maxout(inputs, num_units, axis=None):
    shape = inputs.get_shape().as_list()
    if axis is None:
        # Assume that channel is the last dimension
        axis = -1
    num_channels = shape[axis]
    if num_channels % num_units:
        raise ValueError('number of features({}) is not a multiple of num_units({})'
             .format(num_channels, num_units))
    shape[axis] = -1
    shape += [num_channels // num_units]
    outputs = tf.reduce_max(tf.reshape(inputs, shape), -1, keep_dims=False)
    return outputs

### maxout 적용버전

In [22]:
x_data = x_data = df_x1
sess = tf.Session()
y_data = tf.one_hot(df_y, depth = 4).eval(session=sess)
y_data = tf.reshape(y_data, shape=[-1,4]).eval(session=sess)
print(y_data)
tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, 107])
Y = tf.placeholder(tf.float32, [None, 4])

W1 = tf.get_variable("W1", shape=[107, 64],
                     initializer=xavier_init(107, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.contrib.layers.maxout(tf.matmul(X, W1) + b1, num_units=64)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.contrib.layers.maxout(tf.matmul(L1, W2) + b2, num_units=64)

W3 = tf.get_variable("W3", shape=[64, 4],
                     initializer=xavier_init(64, 4))
b3 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


### tanh 적용버전

In [28]:
x_data = x_data = df_x1
sess = tf.Session()
y_data = tf.one_hot(df_y, depth = 4).eval(session=sess)
y_data = tf.reshape(y_data, shape=[-1,4]).eval(session=sess)
print(y_data)
tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, 107])
Y = tf.placeholder(tf.float32, [None, 4])

W1 = tf.get_variable("W1", shape=[107, 64],
                     initializer=xavier_init(107, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.tanh(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.tanh(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 4],
                     initializer=xavier_init(64, 4))
b3 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


### relu 적용버전

In [48]:
x_data = x_data = df_x1
sess = tf.Session()
y_data = tf.one_hot(df_y, depth = 4).eval(session=sess)
y_data = tf.reshape(y_data, shape=[-1,4]).eval(session=sess)
print(y_data)
tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.002



X = tf.placeholder(tf.float32, [None, 107])
Y = tf.placeholder(tf.float32, [None, 4])

W1 = tf.get_variable("W1", shape=[107, 64],
                     initializer=xavier_init(107, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[64, 4],
                     initializer=xavier_init(64, 4))
b4 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L3, W4) + b4

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


### 실행 코드

In [44]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(20001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 1000 == 0 or step < 100:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: 66.670, 	 Acc: 14.87%
Step:     1, 	 Loss: 31.265, 	 Acc: 12.08%
Step:     2, 	 Loss: 36.665, 	 Acc: 65.29%
Step:     3, 	 Loss: 49.416, 	 Acc: 65.62%
Step:     4, 	 Loss: 53.836, 	 Acc: 65.72%
Step:     5, 	 Loss: 52.145, 	 Acc: 65.72%
Step:     6, 	 Loss: 46.666, 	 Acc: 65.72%
Step:     7, 	 Loss: 38.307, 	 Acc: 65.72%
Step:     8, 	 Loss: 28.040, 	 Acc: 63.17%
Step:     9, 	 Loss: 42.221, 	 Acc: 8.48%
Step:    10, 	 Loss: 32.272, 	 Acc: 8.88%
Step:    11, 	 Loss: 17.086, 	 Acc: 65.42%
Step:    12, 	 Loss: 16.548, 	 Acc: 65.72%
Step:    13, 	 Loss: 14.001, 	 Acc: 65.66%
Step:    14, 	 Loss: 9.776, 	 Acc: 61.65%
Step:    15, 	 Loss: 8.116, 	 Acc: 16.52%
Step:    16, 	 Loss: 9.009, 	 Acc: 15.91%
Step:    17, 	 Loss: 7.979, 	 Acc: 28.39%
Step:    18, 	 Loss: 9.572, 	 Acc: 65.72%
Step:    19, 	 Loss: 11.839, 	 Acc: 65.72%
Step:    20, 	 Loss: 11.671, 	 Acc: 65.72%
Step:    21, 	 Loss: 9.603, 	 Acc: 65.72%
Step:    22, 	 Loss: 8.097, 	 Acc: 28.86%
Step:    23, 	 Loss:

In [47]:
tf.reset_default_graph()

In [45]:
?df.loc